# <center> Web scraping : ISBN </center>

**Objectif :** récupérer des ISBN via du web scraping. On peut ensuite utiliser les ISBN  pour interroger des API pour récupérer le reste des données.
Les ISBN sont ajoutés à un fichier txt qui peut être converti en csv et enregistré.

In [67]:
import os
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

'''arguments de la fonction :
    - number = à partir de quel nombre la recherche commence 
    - etendue = combien de livre le programme doit parcourir
    - txt_file = le nom du fichier txt où seront écrits les ISBN. Si le fichier n'existe pas, alors il sera créé.
'''

def webscraping(number, etendue, txt_file):
    # création d'une liste vide pour recueillir les ISBN
    liste_isbn = []
    
    # boucle pour parcourir chaque livre et récupérer son ISBN
    for x in tqdm(range(etendue), desc="Traitement en cours", dynamic_ncols=True) : 
        # site internet visé
        url = f'https://www.leslibraires.fr/livre/{number}' 
        # récupération de l'isbn
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        div_tab_content = soup.find('dl', class_='dl-horizontal')
        isbn_element = div_tab_content.find('dd', itemprop='isbn')
        
        if isbn_element is None :
            number += 1
            continue
            
        else :
            isbn = isbn_element.text.strip()
            isbn = isbn.replace("-", "")
            number += 1
            
        # on ajoute l'ISBN à la liste
        liste_isbn.append(isbn)
        

    # Vérifier si le fichier existe
    if os.path.isfile(txt_file):
        pass
    else:
        # Créer le fichier s'il n'existe pas
        with open(txt_file, 'w') as f:
            print(f'Le fichier {txt_file} a été créé avec succès')    
    
    # Ouvrir le fichier en mode ajout
    with open(txt_file, 'a') as f:
        # Écrire chaque élément de la liste sur une nouvelle ligne
        f.writelines([element + '\n' for element in liste_isbn])
        
    # message de réussite
    print("Les ISBNs ont bien été écrits dans le fichier.")

In [68]:
webscraping(number=20, etendue=10, txt_file="my_isbn.txt")

Traitement en cours: 100%|█████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.46it/s]

Les ISBNs ont bien été écrits dans le fichier.


## Ouverture du fichier 

In [64]:
import pandas as pd

# Ouvrir le fichier texte et ajouter chaque élément à une liste
with open('my_isbn.txt', 'r') as f:
    liste_isbn = f.read().splitlines()

# Créer un dataframe avec une seule colonne "ISBN"
df = pd.DataFrame({'ISBN': liste_isbn})

# Afficher le dataframe
print(df)

            ISBN
0  9782206080161
1  9780744010121
2  9780140469981
3  9782222966241
4  9782252020111
5  9782747579841
6  9782757301821
7  9782754006101


## Création d'un fichier csv

In [65]:
# Conversion en CSV et enregistrement
df.to_csv('isbn_file.csv', index=False)